In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os


In [ ]:
prefix = '/content/gdrive/My Drive/'
input_path = 'ECE5554_Fall2021/Project/FastDepth_files/'
output_path = 'ECE5554_Fall2021/Project/output/'
sys_path = os.path.join(prefix, input_path)
sys.path.append(sys_path)
output_sys_path = os.path.join(prefix, output_path)
sys.path.append(output_sys_path)
%cd /content/
%cd 'gdrive/My Drive/ECE5554_Fall2021/Project/FastDepth_files/'

In [ ]:
import os
import shutil
import time
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image # PIL is the Python Imaging Library
import cv2
import nyu_transforms
import torch.nn.functional as F

In [ ]:
def convlayer(in_channels,out_channels,kernel_size,stride):
    padding = 1
    assert 2*padding == kernel_size-1, "parameters incorrect. kernel={}, padding={}".format(kernel_size, padding)
    return nn.Sequential(
          nn.Conv2d(in_channels,out_channels,kernel_size,stride=stride,padding=padding,bias=False),
          nn.BatchNorm2d(out_channels),
          nn.ReLU6(inplace=True),
        )

def mobilenetlayer(in_channels,out_channels,kernel_size,stride):
  padding = 1
  return nn.Sequential(
          nn.Conv2d(in_channels,in_channels,kernel_size,stride=stride,padding=padding,bias=False,groups=in_channels),
          nn.BatchNorm2d(in_channels),
          nn.ReLU6(inplace=True),
          nn.Conv2d(in_channels,out_channels,1,1,0,bias=False),
          nn.BatchNorm2d(out_channels),
          nn.ReLU6(inplace=True),
    )
def depthwise(in_channels, kernel_size):
    padding = (kernel_size-1) // 2
    assert 2*padding == kernel_size-1, "parameters incorrect. kernel={}, padding={}".format(kernel_size, padding)
    return nn.Sequential(
          nn.Conv2d(in_channels,in_channels,kernel_size,stride=1,padding=padding,bias=False,groups=in_channels),
          nn.BatchNorm2d(in_channels),
          nn.ReLU(inplace=True),
        )

def pointwise(in_channels, out_channels):
    return nn.Sequential(
          nn.Conv2d(in_channels,out_channels,1,1,0,bias=False),
          nn.BatchNorm2d(out_channels),
          nn.ReLU(inplace=True),
        )

In [ ]:
class Encoder(nn.Module):
  def __init__(self):
        super(Encoder, self).__init__()
        kernel_size = 3
        self.conv0 = convlayer(3,16,kernel_size,2)
        self.conv1 = mobilenetlayer(16,56,kernel_size,1)
        self.conv2 = mobilenetlayer(56,88,kernel_size,2)
        self.conv3 = mobilenetlayer(88,120,kernel_size,1)
        self.conv4 = mobilenetlayer(120,144,kernel_size,2)
        self.conv5 = mobilenetlayer(144,256,kernel_size,1)
        self.conv6 = mobilenetlayer(256,408,kernel_size,2)
        self.conv7 = mobilenetlayer(408,376,kernel_size,1)
        self.conv8 = mobilenetlayer(376,272,kernel_size,1)
        self.conv9 = mobilenetlayer(272,288,kernel_size,1)
        self.conv10 = mobilenetlayer(288,296,kernel_size,1)
        self.conv11 = mobilenetlayer(296,328,kernel_size,1)
        self.conv12 = mobilenetlayer(328,480,kernel_size,2)
        self.conv13 = mobilenetlayer(480,512,kernel_size,1)
  def forward(self, x):
        for i in range(14):
            layer = getattr(self, 'conv{}'.format(i))
            x = layer(x)
            if i==1:
                x1 = x
            elif i==3:
                x2 = x
            elif i==5:
                x3 = x
        return x,x1,x2,x3

In [ ]:
class Decoder(nn.Module):
  def __init__(self):
        super(Decoder, self).__init__()
        kernel_size = 5
        self.decode_conv1 = nn.Sequential(
            depthwise(512, kernel_size),
            pointwise(512, 200))
        self.decode_conv2 = nn.Sequential(
            depthwise(200, kernel_size),
            pointwise(200, 256))
        self.decode_conv3 = nn.Sequential(
            depthwise(256, kernel_size),
            pointwise(256, 120))
        self.decode_conv4 = nn.Sequential(
            depthwise(120, kernel_size),
            pointwise(120, 56))
        self.decode_conv5 = nn.Sequential(
            depthwise(56, kernel_size),
            pointwise(56, 16))
        self.decode_conv6 = pointwise(16, 1)
  def forward(self, x,x1,x2,x3):
        for i in range(1,6):
            layer = getattr(self, 'decode_conv{}'.format(i))
            x = layer(x)
            x = F.interpolate(x, scale_factor=2, mode='nearest')
            if i==4:
                x = x + x1
            elif i==3:
                x = x + x2
            elif i==2:
                x = x + x3
        x = self.decode_conv6(x)
        return x
        

In [ ]:
class DepthEstimation(nn.Module):
  def __init__(self):
        super(DepthEstimation, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
  def forward(self,x):
        x,x1,x2,x3 = self.encoder(x)
        x = self.decoder(x,x1,x2,x3)
        return x

In [ ]:
from PIL import Image
from torchvision import transforms
import pickle
preprocess = transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

In [ ]:
dataset = []
train_dir = "KITTI_Train/"
for i in range(21):
  print(i)  
  pkl_file_name = train_dir+'kitty_dataset_orig_small' +str(i) + '.pkl'
  with open(pkl_file_name, 'rb') as pickle_file:
      dataset_t = pickle.load(pickle_file)
      dataset = dataset + dataset_t

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2)

In [ ]:
print(type(train_loader))
print(len(train_loader))


In [ ]:
def Solver(model, train_loader, optim, criterion, epoch, lr):
  min_loss = 100000000000000
  min_loss_epoch = 0
  for e in range(epoch):
          count = 0
          loss_epoch = 0
          for x, y in train_loader:
            #y = Training_data[filename]
            #input_batch = x.unsqueeze(0) # create a mini-batch as expected by the model
            shape = [y.shape[1],y.shape[2]]
            input_batch = x.to('cuda')
            optim.zero_grad()
            y_pred = model(input_batch) 
            prediction = y_pred.reshape(y_pred.shape[0],y_pred.shape[2],y_pred.shape[3])
            prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=shape,
            mode="bicubic",
            align_corners=False,
            ).squeeze()
            output = prediction.cpu()
            loss = (criterion(output, y))
            if (count % 1000) == 0:
              print(loss)
            loss.backward()
            optim.step()
            loss_epoch += loss.item()
            count += 1
          if loss_epoch <  min_loss:
            min_loss = loss_epoch
            min_loss_epoch = e
            print(str(min_loss) + ' at ' + str(min_loss_epoch))

In [ ]:
lr_variable = 0.0001
epoch = 500
criterion = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=lr_variable)
#optim = torch.optim.SGD(model.parameters(), lr=0.01,momentum=0.9,weight_decay=0.1)
Solver(model,train_loader,optim,criterion,epoch,lr_variable)

In [ ]:
import time
def forward_model_test(raw_img): 
  input_image = Image.open(raw_img).convert('RGB')
  shape = [input_image.size[1],input_image.size[0]]
  #shape = [224,224]
  input_tensor = preprocess(input_image)
  input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
  input_batch = input_batch.to('cuda')
  t1 = time.time()
  prediction = model(input_batch)
  t2 = time.time()
  prediction = prediction.reshape(1,224,224)
  prediction = torch.nn.functional.interpolate(
      prediction.unsqueeze(1),
      size=shape,
      mode="bicubic",
      align_corners=False,
  ).squeeze()

  output = prediction.detach().cpu()
  return output,(t2-t1)

In [ ]:
depth_img_folder = 'KITTI_Depth/2011_09_26_drive_0001_sync/proj_depth/groundtruth/image_02/'
raw_img_folder = 'KITTI_Raw/2011_09_26_drive_0001_sync/image_02/data/'

In [ ]:
filename_raw_t = raw_img_folder+"0000000005.png"
y= forward_model_test(filename_raw_t)
plt.imshow(y)